In [1]:
import h3
import numpy as np
import pandas as pd
import webbrowser
import os
import folium
from folium import Map
from tqmd import tqmd 

In [2]:
def create_map(clusters):
    # Create the map object
    map = Map(tiles="cartodbpositron", 
          attr= 'Roambee - Neel Node Intelligence')

    # Convert the clusters dictionary items to polygons and add them to the map
    for cluster in clusters.values():
        points = cluster['geom']
        # points = [p[::-1] for p in points]
        tooltip = "{0} clustered".format(cluster['count'])
        polygon = folium.vector_layers.Polygon(locations=points, tooltip=tooltip,
                                               fill=True, 
                                               color='#ff0000', 
                                               fill_color='#feb934', 
                                               fill_opacity=0.4, weight=3, opacity=0.4)
        polygon.add_to(map)

    # Determine the map bounding box
    max_lat = df.latitude.max()
    min_lat = df.latitude.min()
    max_lon = df.longitude.max()
    min_lon = df.longitude.min()
    
    # Fit the map to the bounds
    map.fit_bounds([[min_lat, min_lon], [max_lat, max_lon]])
    
    return map
def show_map(map, file_name):
    map.save(file_name)
    wb = webbrowser.open('file://' + os.path.realpath(file_name), new=2)

In [3]:
data =  pd.read_csv("../data/bee_parsed_all.csv")

In [4]:
data.dropna()

,imei,created_date,latitude,longitude,battery,battery_voltage,device_mode,prf,firmware,event_status,is_event,event_name,temperature


In [5]:
data = data.loc[data["latitude"] <1000000]
data = data.loc[data["longitude"] <1000000]

In [6]:
h3_level = 11

In [7]:
def lat_lng_to_h3(row):
    return h3.geo_to_h3(row['latitude'], row['longitude'], h3_level)

In [8]:
data['h3'] = data.apply(lat_lng_to_h3, axis=1)

In [9]:
h3_clusters = dict()

for index, row in data.iterrows():
    key = row['h3']
    if key in h3_clusters:
        h3_clusters[key]["count"] += 1
    else:
        h3_clusters[key] = {"count": 1,
                            "geom": h3.h3_to_geo_boundary(key)}

In [161]:
relevant_clusters = { key:value for (key,value) in h3_clusters.items() if value['count'] >= 50}

In [162]:
h3_map = create_map(relevant_clusters)
show_map(h3_map, "map_h3_{0}.html".format(h3_level))

In [153]:
cluster_geo = [h3.h3_to_geo(h) for h in relevant_clusters.keys()]
cluster_geo

[(51.167926053762365, 6.839661523387771),
 (1.6023491455066743, 103.68362002133416)]